# Objetivos deste trabalho:
- Se familiarizar com o ambiente Notebook e com Python
- Implementar um perceptron simples, treiná-lo no conjunto de TREINO do CIFAR-10 e avaliá-lo no conjunto de TESTE (alvo: distinguir fotos de animais de meios de transporte)
- Utilizar a função sigmóide e verificar seu efeito no treinamento e na avaliação
- Modificar a metodologia para classificar cada classe individualmente (i.e. treinar 10 perceptrons, um para cada classe). Considerar: dado um exemplo, que passará por cada perceptron, como decidir qual é a classe dele?

In [1]:
from __future__ import division

%matplotlib inline

import torch
import torchvision
import numpy as np

np.seterr(all='raise')

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
# Carregar os datasets

dataset_train = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True)

dataset_test = torchvision.datasets.CIFAR10(root='./data', train=False,
                                        download=True)

In [ ]:
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
print(len(dataset_train), len(dataset_test))

In [ ]:
# Converter para tons de cinza

# Treino
grayscale_dataset_train = []
for img,label in dataset_train:
    category = classes[label]
    gray_npimg = np.array(img.convert('L'))
    grayscale_dataset_train.append((gray_npimg,category))
    
# Teste
grayscale_dataset_test = []
for img,label in dataset_test:
    category = classes[label]
    gray_npimg = np.array(img.convert('L'))
    grayscale_dataset_test.append((gray_npimg,category))    

In [ ]:
# Mostrar uma imagem

import matplotlib.pyplot as plt
import numpy as np

image_index = 1
label = grayscale_dataset_train[image_index][1]
npimg = grayscale_dataset_train[image_index][0]

plt.imshow(npimg, cmap='gray')
plt.title(label)
plt.show()

In [ ]:
# Converter para vetores 1D

# Para pensar: por que a divisão por 255 no código abaixo?

# A divisão por 255 (maior valor possível que cada pixel em tom de cinza alcança)
# ocorre para "normalizar os valores". Valores muito altos poderiam causar problemas
# à rede, ou até erros de overflow  nas iterações iniciais, além de exigir que os pesos
# dos perceptrons sejam extremamente precisos em pequena escala, podendo impedir a
# melhoria do modelo por falta de capacidade de representação

linear_dataset_train = []
target_labels = ('plane', 'car', 'ship', 'truck')
for img,category in grayscale_dataset_train:
    linear_img = img.reshape(img.shape[0]*img.shape[1],1) / 255
    if category in target_labels:
        label = 1
    else:
        label = 0
    linear_dataset_train.append((linear_img,label))
    
linear_dataset_test = []
target_labels = ('plane', 'car', 'ship', 'truck')
for img,category in grayscale_dataset_test:
    linear_img = img.reshape(img.shape[0]*img.shape[1],1) / 255
    if category in target_labels:
        label = 1
    else:
        label = 0
    linear_dataset_test.append((linear_img,label))   
    


In [ ]:
size = len(linear_dataset_train[0][0])
print(size)

# Definindo o perceptron

In [ ]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

def perceptron(inputs, weights):
    o = inputs.dot(weights)
    return o

# Treinando o perceptron

In [ ]:
def sigmoid_dx(x):
    y = sigmoid(x)
    return y*(1-y)

def Quantizer(x, threshold=0.5):
    if x > threshold:
        return 1
    return 0
  
def err(target, pred):
    return target - pred

def evaluate(weights, dataset):
    # Defina a métrica de avaliação do perceptron aqui (e.g. acurácia)
    mse,tp,tn,fp,fn = 0,0,0,0,0
    for img,label in dataset:
        inputs = np.append(img, [1])
        o = perceptron(inputs, weights)
        y = sigmoid(o)
        
        mse += (label-y)**2
        result = Quantizer(y)
       
        if result == 1:
            if label == 1: tp+=1
            else: fp+=1
        else: 
            if label == 1: fn+=1
            else: tn+=1        

    return {
        "Accuracy": (tp+tn)/len(dataset),
        "Sensibility": tp/(tp+fn),
        "Specifics": tn/(tn+fp),
        "MSE": mse/len(dataset)
    }

In [ ]:
# Inicialização
weights = (np.random.rand(1,size) - 0.5)[0]
bias = (np.random.rand(1) - 0.5) 
weights = np.append(weights, bias) 

neta = 0.0015

np.random.shuffle(linear_dataset_train)

# Implemente o treino aqui (para separar as duas classes definidas)

accuracies = []

for epoch in range(100):  
    for img, label in linear_dataset_train:
        inputs = np.append(img, [1])
        o = perceptron(inputs, weights)
        y = sigmoid(o)
        loss = neta * inputs * err(label, y) * sigmoid_dx(o)
        weights += loss
        
    results = evaluate(weights, linear_dataset_train)
    accuracies.append(results["Accuracy"])
    print("Epoch ", epoch, ": ", results["Accuracy"])
    
print("Binary Training: ", results)
    

In [ ]:
# Avalie o modelo treinado aqui
# Como a acurácia no conjunto de teste se compara com a acurácia obtida no conjunto de treino?
train_results = evaluate(weights, linear_dataset_train)
print("Train Data Accuracy", train_results)
test_results = evaluate(weights, linear_dataset_test)
print("Test Data Accuracy", test_results)

In [ ]:
# Caso queiram plotar alguma coisa

import matplotlib.pyplot as plt

plt.plot(accuracies)

# Classificando classes individuais

Implemente aqui a modificação do processo de avaliação e treinamento para poder classificar cada classe individualmente.

- Ideia geral: treinar um perceptron por classe (exemplo positivo = exemplos da classe; exemplos negativos = exemplo de todas outras classes)
- Dado um exemplo qualquer, como decidir qual perceptron está dando a classe correta?

In [ ]:
def individual_dataset(target_labels):
  linear_dataset_train = []
  for img,category in grayscale_dataset_train:
      linear_img = img.reshape(img.shape[0]*img.shape[1],1) / 255
      if category in target_labels:
          label = 1
      else:
          label = 0
      linear_dataset_train.append((linear_img,label))


  linear_dataset_test = []
  for img,category in grayscale_dataset_test:
      linear_img = img.reshape(img.shape[0]*img.shape[1],1) / 255
      if category in target_labels:
          label = 1
      else:
          label = 0
      linear_dataset_test.append((linear_img,label))    

  return linear_dataset_train, linear_dataset_test

In [ ]:
modelos = []
for target_label in classes:
  
    linear_dataset_train, linear_dataset_test = individual_dataset(target_label)
    
    weights = (np.random.rand(1,size) - 0.5)[0]
    bias = (np.random.rand(1) - 0.5) 
    weights = np.append(weights, bias) 
    
    neta = 0.0015
    
    np.random.shuffle(linear_dataset_train)

    print("Training", target_label + "...")

    for epoch in range(100):        
        for img, label in linear_dataset_train:
            inputs = np.append(img, [1])
            o = perceptron(inputs, weights)
            y = sigmoid(o)
            loss = neta * inputs * err(label, y) * sigmoid_dx(o)
            weights += loss

        train_results = evaluate(weights, linear_dataset_train) 
        acc = train_results['Accuracy']
    
    
    test_results = evaluate(weights, linear_dataset_test)
    acc_test = test_results['Accuracy']
    modelos.append(
       {
        "Weights": weights,
        "Accu": acc,
        "T_Accu": acc_test,
        "Label": target_label
      }
    )
    print("Accuracy train: ", acc)
    print("Accuracy test: ", acc_test)
    

In [ ]:
corrects = 0
for img,category in grayscale_dataset_test:
    linear_img = np.append((img.reshape(img.shape[0]*img.shape[1],1) / 255), [1])
    
    results = []
    for modelo in modelos:
        o = perceptron(modelo['Weights'], linear_img)
        pred = sigmoid(o)
        results.append((pred, modelo['Label']))
    results = sorted(results, reverse=True)

    if results[0][1] == category:
        corrects += 1

print("General Test with all classes, Acc = ", corrects / len(grayscale_dataset_test))